In [1]:
## Datos
Q = 20 # Caudal en m3/h
d_int = 35.052 # Diámero interno en mm
e = 0.000046 # Rugosidad en m
L = 9 + 16 # Longitud de la tubería en m
k = 2.2 + 7.5 + 0.66 + 1 # Factores k de los accesorios

In [2]:
import numpy as np
from scipy.optimize import root
from numba import jit
from __future__ import division, print_function
d_int_m = d_int / 1000 # Convrsión de diámetro interno en m
Q_m3_s = Q / 3600 # Conversión de Q en m3/s
area = np.pi * (d_int_m / 2) ** 2 # Cálculo de área
velocidad = Q_m3_s / area # Cálculo de velocidad
Re = velocidad * d_int_m / 0.00000113 # Cálculo de Re, se supone agua a 4°C
C2_2g = velocidad ** 2 / (2 * 9.80665) # C^2/(2*g)


## Pérdidas en la tubería
@jit
def colebrook_white(x): # Definición de la ecuación de colebrook-white
    return (2 * np.log10((e / d_int_m) / 3.7 + 2.51 / (Re * np.sqrt(x))) + 1 / np.sqrt(x))

sol = root(colebrook_white,0.02) # Solución numerica de la ecuación colebrook-white
f = sol.x # Se asigna el factor de Darcy-Weisbach
Hf = f * (100 / d_int_m) * (C2_2g) # Se calcula Hf por 100m
Hf_L = f * (L / d_int_m) * (C2_2g) # Se calculan las perdidas a L metros

## Pérdidas en los accesorios
Hf_acc = k * C2_2g

## Pérdidas totales
Hf_tot = Hf_acc + Hf_L

##  Impresion de resultados
print("Velocidad [m/s]: ", velocidad)
print("Número de Reynolds: ", Re)
print("C^2/(2*g): ", C2_2g)
print("Factor de fricción de Darcy-Weisbach: ", f)
print("Pérdidas por 100m: ", Hf)
print("Pérdidas por ", L, "m :", Hf_L)
print("Pérdidas en accesorios : ", Hf_acc)
print("Pérdidas totales : ", Hf_tot)

Velocidad [m/s]:  5.757209157127311
Número de Reynolds:  178585.571128873
C^2/(2*g):  1.6899480086936194
Factor de fricción de Darcy-Weisbach:  [ 0.02230395]
Pérdidas por 100m:  [ 107.53311953]
Pérdidas por  25 m : [ 26.88327988]
Pérdidas en accesorios :  19.197809378759516
Pérdidas totales :  [ 46.08108926]


In [3]:
## Datos
import numpy as np
from scipy.optimize import root
from numba import jit

Q = np.array([2,4,8,10,12,13,15,16,18,8.25]) # Caudal en m3/h (no incluir el cero)
d_int_suc = 40.894 # Diámero interno tubería de succ en mm
d_int_des = 35.052 # Diámero interno tubería de desc en mm
e = 0.000046 # Rugosidad en m
L_suc = 10 # Longitud de la tubería de succión en m
L_desc = 19.5 # Longitud de la tubería de descarga en m
k_suc = 1.56 # Fctores k de los accesorios en la succión
k_desc = 10.36 # Fctores k de los accesorios en la descarga

In [20]:
import pandas as pd
n = Q.size
# Se crean los vectores
velocidad_suc = np.zeros(n)
Re_suc = np.zeros(n)
c2_2g_suc = np.zeros(n)
f_suc = np.zeros(n)
Hf_suc = np.zeros(n)
Hf_L_suc = np.zeros(n)
Hf_acc_suc = np.zeros(n)
Hf_tot_suc = np.zeros(n)
area_suc = np.zeros(n)
velocidad_des = np.zeros(n)
Re_des = np.zeros(n)
c2_2g_des = np.zeros(n)
f_des = np.zeros(n)
Hf_des = np.zeros(n)
Hf_L_des = np.zeros(n)
Hf_acc_des = np.zeros(n)
Hf_tot_des = np.zeros(n)
area_desc = np.zeros(n)


d_int_m_suc = d_int_suc / 1000 # Convrsión de diámetro interno en m
d_int_m_desc = d_int_des / 1000 # Convrsión de diámetro interno en m
Q_m3_s = Q / 3600 # Conversión de Q en m3/s

for i in range(n):
    area_suc[i] = np.pi * (d_int_m_suc / 2) ** 2 # Cálculo de área
    velocidad_suc[i] = Q_m3_s[i] / area_suc[i] # Cálculo de velocidad
    Re_suc[i] = velocidad_suc[i] * d_int_m_suc / 0.00000113 # Cálculo de Re, se supone agua a 4°C
    c2_2g_suc[i] = velocidad_suc[i] ** 2 / (2 * 9.80665) # C^2/(2*g)
    
    ## Pérdidas en la tubería
    @jit
    def colebrook_white(x): # Definición de la ecuación de colebrook-white
        return (2 * np.log10((e / d_int_m_suc) / 3.7 + 2.51 / (Re_suc[i] * np.sqrt(x))) + 1 / np.sqrt(x))
    
    sol = root(colebrook_white,0.03) # Solución numerica de la ecuación colebrook-white
    f_suc[i] = sol.x # Se asigna el factor de Darcy-Weisbach
    Hf_suc[i] = f_suc[i] * (100 / d_int_m_suc) * (c2_2g_suc[i]) # Se calcula Hf por 100m
    Hf_L_suc[i] = f_suc[i] * (L_suc / d_int_m_suc) * (c2_2g_suc[i]) # Se calculan las perdidas a L metros

    ## Pérdidas en los accesorios
    Hf_acc_suc[i] = k_suc * c2_2g_suc[i]

    ## Pérdidas totales
    Hf_tot_suc[i] = Hf_acc_suc[i] + Hf_L_suc[i]
    
    ## Descarga
    area_desc[i] = np.pi * (d_int_m_desc / 2) ** 2 # Cálculo de área
    velocidad_des[i] = Q_m3_s[i] / area_desc[i] # Cálculo de velocidad
    Re_des[i] = velocidad_des[i] * d_int_m_desc / 0.00000113 # Cálculo de Re, se supone agua a 4°C
    c2_2g_des[i] = velocidad_des[i] ** 2 / (2 * 9.80665) # C^2/(2*g)
    
    ## Pérdidas en la tubería
    #@jit
    def colebrook_white(x): # Definición de la ecuación de colebrook-white
        return (2 * np.log10((e / d_int_m_desc) / 3.7 + 2.51 / (Re_des[i] * np.sqrt(x))) + 1 / np.sqrt(x))
    
    sol = root(colebrook_white,0.02) # Solución numerica de la ecuación colebrook-white
    f_des[i] = sol.x # Se asigna el factor de Darcy-Weisbach
    Hf_des[i] = f_des[i] * (100 / d_int_m_desc) * (c2_2g_des[i]) # Se calcula Hf por 100m
    Hf_L_des[i] = f_des[i] * (L_desc / d_int_m_desc) * (c2_2g_des[i]) # Se calculan las perdidas a L metros

    ## Pérdidas en los accesorios
    Hf_acc_des[i] = k_desc * c2_2g_des[i]

    ## Pérdidas totales
    Hf_tot_des[i] = Hf_acc_des[i] + Hf_L_des[i]

tabla = pd.DataFrame({" Q [m3/h]": Q,
                      " Q [m3/s]": Q_m3_s,
                      #"3 Velocidad suc": velocidad_suc,
                      #"4 Re suc": Re_suc,
                      "C^2/2g suc": c2_2g_suc,
                      #"6 f suc": f_suc,
                      "Hf_100 suc": Hf_suc,
                      #"8 Hf a L suc": Hf_L_suc,
                      #"9 Hf acc suc": Hf_acc_suc,
                      "Hf tot suc": Hf_tot_suc,
                      #"11 Velocidad des": velocidad_des,
                      #"12 Re des": Re_des,
                      "C^2/2g des": c2_2g_des,
                      #"14 f des": f_des,
                      "Hf_100 des": Hf_des,
                      #"16 Hf a L des": Hf_L_des,
                      #"17 Hf acc des": Hf_acc_des,
                      "Hf tot des": Hf_tot_des})
tabla 

,Q [m3/h],Q [m3/s],C^2/2g des,C^2/2g suc,Hf tot des,Hf tot suc,Hf_100 des,Hf_100 suc
0,2.00,0.000556,0.016899,0.009122,0.449070,0.080484,1.405086,0.662539
1,4.00,0.001111,0.067598,0.036488,1.677349,0.290382,5.010435,2.334610
2,8.00,0.002222,0.270392,0.145951,6.394540,1.075476,18.427089,8.477929
3,10.00,0.002778,0.422487,0.228048,9.875992,1.648098,28.200134,12.923434
4,12.00,0.003333,0.608381,0.328389,14.104260,2.340108,40.007335,18.278209
5,13.00,0.003611,0.714003,0.385401,16.498260,2.730806,46.672762,21.295801
6,15.00,0.004167,0.950596,0.513108,21.845747,3.601487,61.526028,28.010383
7,16.00,0.004444,1.081567,0.583803,24.799167,4.081441,69.713517,31.707082
8,18.00,0.005000,1.368858,0.738875,31.265234,5.130521,87.609571,39.778753
9,8.25,0.002292,0.287555,0.155215,6.788851,1.140513,19.537328,8.983777


In [21]:
H = x + c2_2g_des + Hf_tot_des + Hf_tot_suc

In [22]:
H

array([ 22.57647818,  24.0653532 ,  29.77043199,  33.97660092,
        39.08277362,  41.97309289,  48.42785394,  51.9921985 ,
        59.79463705,  30.24694366])